In [2]:
#this script will take a set of Cq values and a properly formatted PCR plan for a 384 well plate and output Cq values next to their appropiate well label
#this is the development script. The final version should be run from the command line and is located elsewhere.

#imports we need
import pandas as pd

def main_code():
    #reads in raw Cq values and takes only the Cq column
    cq = pd.read_excel('samplecq.xlsx')
    cq = cq.set_index(['Well'])
    cq = cq['Cq']

    #reads in formatted qPCR plan
    plan = pd.read_excel('sampleplan.xlsx')

    #formats the plan dataframe so each label includes the target gene
    for i in range(0,15):
        gene = plan.iloc[i,24]
        name = plan.iloc[i]
        plan.iloc[i] = gene + ': ' + name
    #discard the gene col as this just gets in the way now
    plan = plan.drop(['Gene'], axis=1)

    #this loop will combine the plan and cq dataframes by appending to a list named 'res' and then converting that into a dataframe named 'combined'
    res = []
    col_counter = 0
    for j in range(0,16):
        letter = plan.index[j]
        for i in range(0,24):
            res.append([plan.at[letter,i+1], cq.iloc[i+col_counter]])
        col_counter = col_counter+24
    combined = pd.DataFrame(res)
    combined = combined.rename(index=str, columns={0:'gene',1:'Cq'})
    combined = combined.set_index(cq.index)
    #save the results!
    combined.to_csv('combined.csv')

def check_output():
    #quick and dirty check to make sure cq values in the original cq results match the same spot in the final 'combined' dataframe
    #NaN Cq values will always return a False.
    for i in range(0,384):
        print(i, cq.iloc[i] == combined.iloc[i,1])

main_code()